In [3]:
from datetime import datetime
from pathlib import Path

import cftime
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray  # noqa: F401
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
import httpx
from tqdm import tqdm

def download_file(url, local_path):
    with httpx.Client(timeout=None) as client:
        with client.stream("GET", url) as response:
            response.raise_for_status()

            total = int(response.headers.get("Content-Length", 0))
            if total == 0:
                print("No content length header. Progress bar may not be accurate.")

            with open(local_path, "wb") as f, tqdm(
                total=total, unit="B", unit_scale=True, desc="Downloading"
            ) as pbar:
                for chunk in response.iter_bytes(chunk_size=1024 * 1024):  # 1 MB chunks
                    f.write(chunk)
                    pbar.update(len(chunk))

    print(f" Download complete: {local_path}")




In [10]:
local_path = f"CSAT_NCF_CEMVN_combined_{datetime.today().strftime('%Y-%m-%d')}.zip"
local_path

'CSAT_NCF_CEMVN_combined_2025-07-01.zip'

In [15]:

download_file(
    "https://chldata.erdc.dren.mil/thredds/fileServer/csat/CSAT_NCF_CEMVN_combined.zip",
    local_path)

Downloading: 100%|██████████| 8.30G/8.30G [54:08<00:00, 2.56MB/s]  


✅ Download complete: CSAT_NCF_CEMVN_combined_2025-07-01.zip


In [16]:
import zipfile
import os

# Create the output directory if it doesn't exist
output_dir = Path("data/original")
output_dir.mkdir(parents=True, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(local_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

In [17]:



def get_reach_properties(reach_ID=None, returnGeo=False):
    """
    Retrieves select attributes from the National Channel Framework for a given ReachID.
    Parameters
    ----------
    reach_ID : str, optional
        The channel reach ID to query (expected format: CEXYZ...)
    returnGeo : bool, default False
        Whether to include geometry in the returned GeoDataFrame

    Returns
    -------
    geopandas.GeoDataFrame
        Contains attributes of the specified reach including:
        - channelreachidpk
        - sdsfeaturename
        - sdsfeaturedescription
        - sourceprojection
        - channelareaidfk
        - depthmaintained
        - depthauthorized
        - geometry (if returnGeo=True)
    Notes
    -----
    This function queries the ArcGIS REST API for the National Channel Framework.
    """

    # TODO Check if reach_ID is a valid format (starts with CEXYZ)
    query = (
        f"https://services7.arcgis.com/n1YM8pTrFmm7L4hs/ArcGIS/rest/services/National_Channel_Framework/"
        f"FeatureServer/2/query?where=channelreachidpk='{reach_ID}'&outFields=channelreachidpk,sdsfeaturename,"
        f"sdsfeaturedescription,sourceprojection,channelareaidfk,depthmaintained,depthauthorized&returnGeometry={str(returnGeo)}&f=json&token="
    )

    gdf = gpd.read_file(query)

    return gdf


def clean_csat_netcdf(district5, reachID):
    """
    Clean and transform CSAT (Corps Shoaling Analysis Tool) NetCDF data.
    This function processes raw CSAT NetCDF files by cleaning the data, converting formats,
    and restructuring the data from 2D to 3D arrays. It handles temporal conversions,
    extreme value filtering, and proper NetCDF attribute assignments.

    Parameters
    ----------
    district5 : str
        The district identifier for the data location
    reachID : str
        The reach identifier for the specific waterway segment

    Returns
    -------
    None
        The function saves the processed NetCDF file to './data/updated/{district5}/{reachID}.nc'
    Notes
    -----
    The function performs the following operations:
    - Trims excess points from 2D arrays
    - Converts time variables to conventional format
    - Replaces extreme float values with NaN
    - Converts data structure from 2D to 3D
    - Adds proper CF-compliant metadata
    - Applies CRS information from reach properties
    - Compresses the output file
    The output NetCDF file follows CF-1.11 conventions and includes standardized
    attributes for depth, time, and spatial coordinates.

    Example
    -------
    >>> clean_csat_netcdf('CEMVN', 'SW_01_SWP_01')
    """

    sample_path = Path(
        rf"./data/original/{district5}/{reachID}.nc"
    )  # TODO: Update path to match your directory structure, this is where the files downloaded from the CIRP website are stored
    print(rf"{district5}/{reachID}.nc")

    with xr.open_dataset(sample_path, chunks = {"x":256,"y":256}) as ds:
        # --- Trimming 2D array ---
        # The excess points can be trimmed by eliminating extreme values in the `points` variable
        PointIdx = np.where(ds["points"] >= 0)[0]
        print(f"\t{len(PointIdx):,} out of {len(ds['points']):,} points retained")

        # ds['elevations'].isel(points=PointIdx)
        ds2 = ds.isel(points=PointIdx).copy()

    # --- Converting time variable to a conventional format ---
    # Use Pandas to parse original time (floats) and create datetime.datetime objects
    time_var = ds2["time"].astype(int).astype(str)  # .values.astype(int).astype(str))
    time_var = pd.to_datetime(time_var)
    time_dt = time_var.to_pydatetime()

    # use cftime date2num function to transform datetime.datetime object
    time_datum = datetime(1900, 1, 1)
    time_units = f"days since {time_datum.year}-{time_datum.month}-{time_datum.day}"
    time_vals = cftime.date2num(time_dt, time_units)

    # --- Replacing extreme float values with nan-values ---
    ds2["elevations"] = ds2["elevations"].where(ds2["elevations"] > -9999)

    # --- Converting from 2D to 3D ---
    uLat, latIndices = np.unique(ds2["latitudes"], return_inverse=True)
    uLon, lonIndices = np.unique(ds2["longitudes"], return_inverse=True)
    nTimes = len(time_vals)

    print(f"\tn_uLat={len(uLat)}, n_uLon={len(uLon)}, nTimes={nTimes}")

    elevs_3D = np.full((nTimes, len(uLat), len(uLon)), fill_value=np.nan)

    for i in np.arange(nTimes):
        elevs_3D[i, latIndices, lonIndices] = ds2["elevations"].isel(time=i).values

    ds3 = xr.Dataset(
        {
            "depth": (["time", "y", "x"], elevs_3D),
        },
        coords={
            "time": time_vals,
            "y": (["y"], uLat),
            "x": (["x"], uLon),
            "reference_time": pd.Timestamp("1900-01-01"),
        },
    )

    # Adding global attribute metadata
    ds3.attrs["Conventions"] = "CF-1.11"
    ds3.attrs["Title"] = "Corps Shoaling Analysis Tool Input Data - Test"
    ds3.attrs["description"] = (
        "This file collates the bathymetry data provided from the USACE eHydro program.  Data is collated by reach in a given USACE navigation channel."
    )

    # Adding attributes to data variables
    ds3.depth.attrs["standard_name"] = "depth"
    ds3.depth.attrs["long_name"] = "depth below datum"
    ds3.depth.attrs["units"] = "US_survey_feet"

    ds3.time.attrs["standard_name"] = "time"
    ds3.time.attrs["long_name"] = "time"
    ds3.time.attrs["units"] = time_units
    ds3.time.encoding["units"] = time_units

    ds3.x.attrs["axis"] = "X"  # Optional
    ds3.x.attrs["standard_name"] = "projection_x_coordinate"
    ds3.x.attrs["long_name"] = "x-coordinate in projected coordinate system"

    ds3.y.attrs["axis"] = "Y"  # Optional
    ds3.y.attrs["standard_name"] = "projection_y_coordinate"
    ds3.y.attrs["long_name"] = "y-coordinate in projected coordinate system"

    # Query NCF REST API for ReachID projection information
    sample_gdf = get_reach_properties(reach_ID=f"{district5}_{reachID}", returnGeo=True)

    ds3 = ds3.rio.write_crs(input_crs=sample_gdf.sourceprojection[0])

    comp = 5  # compression level
    encoding_dict = {
        "depth": {
            "zlib": True,
            "complevel": comp,
        },
        "y": {
            "zlib": True,
            "complevel": comp,
        },
        "x": {
            "zlib": True,
            "complevel": comp,
        },
    }

    output_filename = Path(rf"../../Desktop/Data_prep_SWP/data/updated/{district5}/{reachID}.nc")  # TODO: Update path to match your directory structure
    ds3.to_netcdf(output_filename, encoding=encoding_dict)


# Convert NetCDF files from 2D to 3D
district5 = "CEMVN"
reach_list = [p.stem for p in Path(rf"../../Desktop/Data_prep_SWP/data/original/{district5}").glob("SW_*")]  # TODO: Update path to match your directory structure

for r in reach_list:
    clean_csat_netcdf(district5, r)





CEMVN/SW_01_SWP_01.nc
	112,971 out of 1,119,625 points retained
	n_uLat=1325, n_uLon=844, nTimes=640
CEMVN/SW_02_SWP_01.nc
	111,904 out of 1,210,833 points retained
	n_uLat=1027, n_uLon=1178, nTimes=639
CEMVN/SW_03_SWP_01.nc
	110,820 out of 1,184,964 points retained
	n_uLat=1146, n_uLon=1034, nTimes=1796
CEMVN/SW_04_SWP_01.nc
	110,759 out of 1,035,368 points retained
	n_uLat=1323, n_uLon=782, nTimes=2383
CEMVN/SW_05_SWP_01.nc
	121,510 out of 530,136 points retained
	n_uLat=1592, n_uLon=333, nTimes=2969
CEMVN/SW_06_SWP_01.nc
	111,279 out of 880,376 points retained
	n_uLat=1393, n_uLon=632, nTimes=2047
CEMVN/SW_07_SWP_01.nc
	114,651 out of 1,178,712 points retained
	n_uLat=1284, n_uLon=918, nTimes=1967
CEMVN/SW_08_SWP_01.nc
	113,589 out of 1,156,176 points retained
	n_uLat=1302, n_uLon=888, nTimes=1823
CEMVN/SW_09_SWP_01.nc
	112,886 out of 1,177,394 points retained
	n_uLat=1277, n_uLon=922, nTimes=1916
CEMVN/SW_10_SWP_01.nc
	112,527 out of 1,178,980 points retained
	n_uLat=1265, n_uLon=9

In [ ]:
import json
import os

def remove_typesize_from_zmetadata(zarr_store_path):
    zmeta_file = os.path.join(zarr_store_path, ".zmetadata")

    if not os.path.exists(zmeta_file):
        print(f"No .zmetadata file found at {zmeta_file}")
        return

    with open(zmeta_file, "r") as f:
        meta = json.load(f)

    # Recursively remove all "typesize" keys
    def remove_typesize(obj):
        if isinstance(obj, dict):
            return {k: remove_typesize(v) for k, v in obj.items() if k != "typesize"}
        elif isinstance(obj, list):
            return [remove_typesize(i) for i in obj]
        return obj

    cleaned_meta = remove_typesize(meta)

    with open(zmeta_file, "w") as f:
        json.dump(cleaned_meta, f, indent=2)

    print(f"Removed 'typesize' from .zmetadata in: {zarr_store_path}")


In [ ]:
# Convert from NetCDF to Zarr format
district5 = "CEMVN"
reachID_list = [f"SW_{x + 1:02}_SWP_01" for x in range(0, 13)]  # TODO: Update to match your range of reach IDs

for reachID in reachID_list:
    print(reachID)

    # Load SWP netcdf file
    single_nc = Path(rf"../../Desktop/Data_prep_SWP/data/updated/{district5}/{reachID}.nc")  # TODO: Update path to match your directory structure
    ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks

    # Demonstrate existance of duplicate dates
    t = ds.time.data
    print(f"\tOriginal dates: {len(t)}")
    print(f"\tReduced dates: {len(np.unique(t))}")

    # Merge duplicate dates
    da = ds.depth.groupby("time").reduce(np.nanmean, keep_attrs=True)
    ds.close()
    # rechunk the data for better timeseries analysis
    da = da.chunk(dict(time=-1, x=100, y=100))

    # fill NaN values (interpolation will error if NaNs are present)
    da2 = da.interpolate_na(dim="time", method="linear")

    # interpolate to a daily time series
    da_fullrange = da2.interp(time=pd.date_range(da2.time.min().values, da2.time.max().values))

    # --- Prepare output for storage ---

    # convert from DataArray to Dataset
    ds_fullrange = da_fullrange.to_dataset()

    # copy original attributes for dataset and variables
    ds_fullrange.attrs = ds.attrs
    ds_fullrange["spatial_ref"] = ds["spatial_ref"]
    ds_fullrange["time"].attrs = ds["time"].attrs

    # set output path for zarr folder
    output_dir = Path(rf"../../Desktop/Data_prep_SWP/data/interpolated/{district5}/{reachID}_{datetime.today().strftime('%Y-%m-%d')}.zarr")  # TODO: Update path to match your directory structure
    # write to disk (Zarr)
    print("\tSaving Zarr file")
    ds_fullrange.to_zarr(
    store=output_dir,
    mode="w",
    consolidated=True,
    zarr_format=2
   )
    
    remove_typesize_from_zmetadata(output_dir)


SW_01_SWP_01
	Original dates: 637
	Reduced dates: 542


C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks
C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks


	Saving Zarr file
Removed 'typesize' from .zmetadata in: ..\..\Desktop\Data_prep_SWP\data\interpolated\CEMVN\SW_01_SWP_01_2025-06-23.zarr
SW_02_SWP_01
	Original dates: 634
	Reduced dates: 616


C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks
C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks


	Saving Zarr file
Removed 'typesize' from .zmetadata in: ..\..\Desktop\Data_prep_SWP\data\interpolated\CEMVN\SW_02_SWP_01_2025-06-23.zarr
SW_03_SWP_01
	Original dates: 1793
	Reduced dates: 1587


...

	Saving Zarr file
Removed 'typesize' from .zmetadata in: ..\..\Desktop\Data_prep_SWP\data\interpolated\CEMVN\SW_11_SWP_01_2025-06-23.zarr
SW_12_SWP_01
	Original dates: 1678
	Reduced dates: 1429


C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks
C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks


	Saving Zarr file
Removed 'typesize' from .zmetadata in: ..\..\Desktop\Data_prep_SWP\data\interpolated\CEMVN\SW_12_SWP_01_2025-06-23.zarr
SW_13_SWP_01
	Original dates: 322
	Reduced dates: 203
	Saving Zarr file


C:\Users\jbroders\AppData\Local\Temp\ipykernel_43380\3340438995.py:10: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 256. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(single_nc, chunks={"y": 256, "x": 256})  # specify initial chunks


Removed 'typesize' from .zmetadata in: ..\..\Desktop\Data_prep_SWP\data\interpolated\CEMVN\SW_13_SWP_01_2025-06-23.zarr


In [ ]:
import xarray as xr
import pandas as pd
from datetime import datetime

reachID_list = [f"SW_{x + 1:02}_SWP_01" for x in range(0, 13)]

results = []

for reach in reachID_list:
    try:
        zarr_path = rf"../../Desktop/Data_prep_SWP/data/interpolated/CEMVN/{reach}_{datetime.today().strftime('%Y-%m-%d')}.zarr"
        zarr_array = xr.open_zarr(zarr_path)
        last_date = zarr_array['time'][-1].values
        last_date = pd.to_datetime(last_date).date()
        results.append((reach, last_date))
        print(f"Last date for {reach}: {last_date}")
        zarr_array.close()
    except Exception as e:
        print(f"Error reading {reach}: {e}")
        results.append((reach, None))  # Log as missing

# Create and save DataFrame
df = pd.DataFrame(results, columns=["ReachID", "LastSurveyDate"])
df.to_csv(f"reach_last_survey_dates_{datetime.today().strftime('%Y-%m-%d')}.csv", index=False)
print("CSV saved")


Last date for SW_01_SWP_01: 2025-05-27
Last date for SW_02_SWP_01: 2025-05-28
Last date for SW_03_SWP_01: 2025-05-28
Last date for SW_04_SWP_01: 2025-05-29
Last date for SW_05_SWP_01: 2025-06-02
Last date for SW_06_SWP_01: 2025-05-29
Last date for SW_07_SWP_01: 2025-06-02
Last date for SW_08_SWP_01: 2025-06-02
Last date for SW_09_SWP_01: 2025-05-30
Last date for SW_10_SWP_01: 2025-06-01
Last date for SW_11_SWP_01: 2025-06-02
Last date for SW_12_SWP_01: 2025-05-28
Last date for SW_13_SWP_01: 2025-05-23
CSV saved


In [ ]:
"""def dredge_percent_collection(depth,date):
    dredge_percent=[]
    threshold_deepening = 50
    threshold_pre_deepening = 45
    combined= [(pos, time) for pos, time in zip(depth, date)]
    for item in combined:
        if item[1]< np.datetime64('2020-01-01'):
            dredge = (item[0]< threshold_pre_deepening).sum()/(~np.isnan(item[0])).sum()
            dredge_percent.append([dredge,item[1]])
        else:
            dredge = (item[0]< threshold_deepening).sum()/(~np.isnan(item[0])).sum()
            dredge_percent.append([dredge,item[1]])
    return dredge_percent
    """

"def dredge_percent_collection(depth,date):\n    dredge_percent=[]\n    threshold_deepening = 50\n    threshold_pre_deepening = 45\n    combined= [(pos, time) for pos, time in zip(depth, date)]\n    for item in combined:\n        if item[1]< np.datetime64('2020-01-01'):\n            dredge = (item[0]< threshold_pre_deepening).sum()/(~np.isnan(item[0])).sum()\n            dredge_percent.append([dredge,item[1]])\n        else:\n            dredge = (item[0]< threshold_deepening).sum()/(~np.isnan(item[0])).sum()\n            dredge_percent.append([dredge,item[1]])\n    return dredge_percent\n    "

In [ ]:
"""
plt.figure(figsize=(10, 6))
plt.boxplot(box_data, vert=False, positions=range(1, len(box_data) + 1),showfliers=False)
plt.yticks(ticks=range(1, len(labels) + 1), labels=labels)
plt.xlabel("Percent Above Threshold")
plt.title("Percent Above by Reach")
plt.tight_layout()
plt.show()
"""

'\nplt.figure(figsize=(10, 6))\nplt.boxplot(box_data, vert=False, positions=range(1, len(box_data) + 1),showfliers=False)\nplt.yticks(ticks=range(1, len(labels) + 1), labels=labels)\nplt.xlabel("Percent Above Threshold")\nplt.title("Percent Above by Reach")\nplt.tight_layout()\nplt.show()\n'

In [ ]:
#Day before dredge percent

